In [1]:
import pandas as pd
import numpy as np

In [2]:
courier_actions = pd.read_csv("courier_actions.csv")

In [3]:
courier_actions.head()

,courier_id,order_id,action,time
0,12,1,accept_order,24/08/22 01:52
1,12,1,deliver_order,24/08/22 02:15
2,156,2,accept_order,24/08/22 06:37
3,156,2,deliver_order,24/08/22 06:56
4,49,3,accept_order,24/08/22 07:35


In [4]:
courier_actions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9998 entries, 0 to 9997
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   courier_id  9998 non-null   int64 
 1   order_id    9998 non-null   int64 
 2   action      9998 non-null   object
 3   time        9998 non-null   object
dtypes: int64(2), object(2)
memory usage: 312.6+ KB


In [5]:
# Alteração na tabela para combinar os tempos de aceitar e entregar para cada pedido
courier_actions = courier_actions.pivot(index=['courier_id', 'order_id'], columns='action', values='time').reset_index()

# Renomear as colunas
courier_actions.columns = ['courier_id', 'order_id', 'accept_time', 'deliver_time']

# Ordernar pelo id do pedido
courier_actions = courier_actions.sort_values(by='order_id').reset_index()

# Remover a antiga coluna de index
del courier_actions['index']

# Remover valores NaN e infinitos (inf)
courier_actions = courier_actions.replace([np.inf, -np.inf], np.nan).dropna()

# Converter as colunas em objetos de data e tempo
courier_actions['accept_time'] = pd.to_datetime(courier_actions['accept_time'], format='%d/%m/%y %H:%M')
courier_actions['deliver_time'] = pd.to_datetime(courier_actions['deliver_time'], format='%d/%m/%y %H:%M')

# Calcular a diferença de tempo entre accept_time e deliver_time
courier_actions['waiting_time'] = courier_actions['deliver_time'] - courier_actions['accept_time']

# Converter a diferença de tempo em minutos (número inteiro)
courier_actions['waiting_time_minutes'] = (courier_actions['waiting_time'].dt.total_seconds() / 60).astype(int)

# Exibir o novo DataFrame com as alterações
courier_actions.head()

,courier_id,order_id,accept_time,deliver_time,waiting_time,waiting_time_minutes
0,12,1,2022-08-24 01:52:00,2022-08-24 02:15:00,0 days 00:23:00,23
1,156,2,2022-08-24 06:37:00,2022-08-24 06:56:00,0 days 00:19:00,19
2,49,3,2022-08-24 07:35:00,2022-08-24 07:54:00,0 days 00:19:00,19
3,134,4,2022-08-24 10:39:00,2022-08-24 10:58:00,0 days 00:19:00,19
4,106,5,2022-08-24 12:34:00,2022-08-24 12:59:00,0 days 00:25:00,25


In [6]:
courier_actions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4876 entries, 0 to 5121
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   courier_id            4876 non-null   int64          
 1   order_id              4876 non-null   int64          
 2   accept_time           4876 non-null   datetime64[ns] 
 3   deliver_time          4876 non-null   datetime64[ns] 
 4   waiting_time          4876 non-null   timedelta64[ns]
 5   waiting_time_minutes  4876 non-null   int32          
dtypes: datetime64[ns](2), int32(1), int64(2), timedelta64[ns](1)
memory usage: 247.6 KB


In [7]:
courier_actions.describe()

,courier_id,order_id,waiting_time,waiting_time_minutes
count,4876.000000,4876.000000,4876,4876.000000
mean,316.656071,2567.144176,0 days 00:19:53.601312551,19.893355
std,210.075954,1482.820508,0 days 00:03:01.896764720,3.031613
min,1.000000,1.000000,0 days 00:08:00,8.000000
25%,139.000000,1277.750000,0 days 00:18:00,18.000000
50%,293.000000,2570.500000,0 days 00:20:00,20.000000
75%,471.000000,3856.250000,0 days 00:22:00,22.000000
max,1000.000000,5122.000000,0 days 00:31:00,31.000000


In [8]:
courier_actions.to_csv("courier_actions_new.csv", index=False)

In [9]:
# Convertendo a coluna 'accept_time' para o tipo datetime
courier_actions['accept_time'] = pd.to_datetime(courier_actions['accept_time'])

# Definindo a coluna 'accept_time' como o índice do DataFrame
courier_actions.set_index('accept_time', inplace=True)

# Calculando a quantidade de pedidos por semana
pedidos_por_mes = courier_actions.resample('M').count()

pedidos_por_mes

,courier_id,order_id,deliver_time,waiting_time,waiting_time_minutes
accept_time,,,,,
2022-08-31,4876,4876,4876,4876,4876


In [10]:
dictionary = {'a': 1, 'b': 2}
len(dictionary)

2